In [1]:
!pip install pyspark
import pyspark

In [2]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('Logistic Regression').getOrCreate()

In [3]:
from pyspark.ml.classification import LogisticRegression


In [4]:
df = spark.read.format("libsvm").load("C:/Users/Harshini/Downloads/sample_libsvm_data.txt")

In [5]:
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)


In [6]:
lrModel = lr.fit(df)


In [7]:
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))


Coefficients: (692,[272,300,323,350,351,378,379,405,406,407,428,433,434,435,455,456,461,462,483,484,489,490,496,511,512,517,539,540,568],[-7.52068987138421e-05,-8.115773146847101e-05,3.814692771846369e-05,0.0003776490540424337,0.00034051483661944103,0.0005514455157343105,0.0004085386116096913,0.000419746733274946,0.0008119171358670028,0.0005027708372668751,-2.3929260406601844e-05,0.000574504802090229,0.0009037546426803721,7.818229700244018e-05,-2.1787551952912764e-05,-3.4021658217896256e-05,0.0004966517360637634,0.0008190557828370367,-8.017982139522704e-05,-2.7431694037836214e-05,0.0004810832226238988,0.00048408017626778765,-8.926472920011488e-06,-0.00034148812330427335,-8.950592574121486e-05,0.00048645469116892167,-8.478698005186209e-05,-0.0004234783215831763,-7.29653577763134e-05])
Intercept: -0.5991460286401435


In [8]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer,VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [9]:
df = spark.read.format("libsvm").load("C:/Users/Harshini/Downloads/sample_libsvm_data.txt")

In [12]:
labelIndexer=StringIndexer(inputCol="label",outputCol="indexedLabel").fit(df)

In [18]:
import warnings
warnings.simplefilter("ignore")

In [56]:
featureIndexer=VectorIndexer(inputCol="features",outputCol="indexedFeatures",maxCategories=4).fit(df)

In [57]:
trainingData, testData = df.randomSplit([0.7, 0.3])

In [58]:
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")


In [59]:
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])


In [60]:
model = pipeline.fit(trainingData)


In [61]:
predictions = model.transform(testData)


In [62]:
predictions.select("prediction", "indexedLabel", "features").show(5)


+----------+------------+--------------------+
|prediction|indexedLabel|            features|
+----------+------------+--------------------+
|       1.0|         1.0|(692,[95,96,97,12...|
|       1.0|         1.0|(692,[98,99,100,1...|
|       1.0|         1.0|(692,[122,123,124...|
|       1.0|         1.0|(692,[122,123,148...|
|       1.0|         1.0|(692,[123,124,125...|
+----------+------------+--------------------+
only showing top 5 rows



In [68]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(accuracy)


1.0


In [64]:
print("Test Error = %g " % (1.0 - accuracy))


Test Error = 0 


In [66]:
treeModel = model.stages[2]
# summary only
print(treeModel)

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_75995946513e, depth=2, numNodes=5, numClasses=2, numFeatures=692
